In [ ]:
import os
import sys
import yaml

import numpy as np
import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

from lasso import Project

from lasso import bicounty

import pickle

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

## Remote I/O

In [ ]:
root_dir = ""
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_04')
lasso_dir = os.path.join(f"{root_dir}/software/Lasso")

In [ ]:
parameters = Parameters(lasso_base_dir = lasso_dir)

# Load Scenario

In [ ]:
pickle_file_name = f"{input_dir}/version_03_scenario.pickle"
curr_scenario = pickle.load(open(pickle_file_name, 'rb'))

## Make Travel Model Network

In [ ]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = curr_scenario.road_net, 
    parameters = parameters
)

In [ ]:
model_net.nodes_mtc_df = model_net.nodes_df.copy()
model_net.links_mtc_df = model_net.links_df.copy()

## Write to disk

### Write model network as geopackage

In [ ]:
# MODEL_ROADWAY_LINK_VARIABLES = [
#   'A','B','model_link_id','shstGeometryId','name','roadway',                    # IDs
#   'ft','assignable','cntype','distance','county',                               # Misc attributes
#   'bike_access','drive_access','walk_access','rail_only','bus_only','transit',  # Mode attributes
#   'managed','tollbooth','tollseg','segment_id',                                 # Managed roadway
#   'lanes_EA','lanes_AM','lanes_MD','lanes_PM','lanes_EV',                       # Lanes
#   'useclass_EA','useclass_AM','useclass_MD','useclass_PM','useclass_EV',        # Use classes
#   'nmt2010', 'nmt2020',                                                         # bicounty nm variable  
#   'geometry'                                                                    # geometry
# ]

# MODEL_ROADWAY_NODE_VARIABLES = [
#   'N','osm_node_id','tap_id',                                                   # IDs
#   'county',                                                                     # Misc attributes
#   'bike_access','drive_access','walk_access','rail_only','farezone', 'pnr',     # Mode attributes
#   'geometry'                                                                    # geometry
# ]

In [ ]:
# model_net.write_roadway_as_shp(
#     output_dir= os.path.join(output_dir, 'model_networks', 'geopackages'),
#     link_output_variables = MODEL_ROADWAY_LINK_VARIABLES,
#     node_output_variables = MODEL_ROADWAY_NODE_VARIABLES,
#     output_gpkg = 'model_net.gpkg',
#     output_link_gpkg_layer = 'roadway_links',
#     output_node_gpkg_layer = 'roadway_nodes',
#     #output_gpkg_link_filter = args.gpkg_link_filter
# )

### Write model network for Cube

In [ ]:
model_net.write_roadway_as_fixedwidth(
    output_dir = os.path.join(output_dir, 'model_networks'),
    output_link_txt = 'links.txt',
    output_node_txt = 'nodes.txt',
    output_link_header_width_txt = 'links_header_width.txt',
    output_node_header_width_txt = 'nodes_header_width.txt',
    output_cube_network_script = 'make_complete_network_from_fixed_width_file.s',
    #drive_only = True
)

### write pickle file

In [ ]:
working_scenario_filename = os.path.join(output_dir, 'scenario_2020.pickle')
pickle.dump(curr_scenario, open(working_scenario_filename, 'wb'))

### write to standard format

In [ ]:
# model_net.write(path = os.path.join(output_dir, 'standard_networks'))